In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
import cv2
from tkinter import Tk, filedialog, Button, Label, Toplevel

# Load the trained model
model = tf.keras.models.load_model('insect_classifier.h5')

# Data generators to get class labels
train_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)
train_generator = train_datagen.flow_from_directory(
    'data',
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical',
    subset='training'
)

def classify_image(image_path):
    try:
        img = cv2.imread(image_path)
        if img is None:
            print("Error: Unable to read the image file.")
            return None
        img = cv2.resize(img, (150, 150)) / 255.0  # Resize the image to (150, 150)
        img_array = np.expand_dims(img, axis=0)
        prediction = model.predict(img_array)
        return np.argmax(prediction, axis=1)[0]
    except Exception as e:
        print(f"Error in classify_image: {e}")
        return None

def classify_real_time():
    cap = cv2.VideoCapture(0)
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        img_array = cv2.resize(frame, (150, 150)) / 255.0  # Resize the frame to (150, 150)
        img_array = np.expand_dims(img_array, axis=0)
        prediction = model.predict(img_array)
        class_idx = np.argmax(prediction, axis=1)[0]
        class_label = list(train_generator.class_indices.keys())[class_idx]
        
        cv2.putText(frame, class_label, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
        cv2.imshow('Real-Time Insect Detection', frame)
        
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

def upload_and_classify():
    def reupload_image():
        file_path = filedialog.askopenfilename()
        if file_path:
            print(f"Selected file: {file_path}")
            class_idx = classify_image(file_path)
            if class_idx is not None:
                class_label = list(train_generator.class_indices.keys())[class_idx]
                print(f'The uploaded image is classified as: {class_label}')
                img = cv2.imread(file_path)
                cv2.putText(img, class_label, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
                cv2.imshow('Uploaded Image Classification', img)
                cv2.waitKey(0)
                cv2.destroyAllWindows()
            else:
                print("Error: Image classification failed.")
        else:
            print("Error: No file selected.")

    root = Toplevel()
    root.withdraw()
    reupload_image()

def show_gui():
    root = Tk()
    root.title("Insect Detection")

    def on_real_time_click():
        root.destroy()
        classify_real_time()

    def on_upload_click():
        upload_and_classify()

    label = Label(root, text="Choose detection mode:")
    label.pack(pady=10)

    real_time_button = Button(root, text="Real-Time Detection", command=on_real_time_click)
    real_time_button.pack(pady=10)

    upload_button = Button(root, text="Upload Image", command=on_upload_click)
    upload_button.pack(pady=10)

    root.mainloop()

if __name__ == '__main__':
    show_gui()


Found 3561 images belonging to 5 classes.
Selected file: C:/Users/Asus/Documents/PhD/Projects/Insect PhD/data/Butterfly/google57.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 653ms/step
The uploaded image is classified as: Butterfly
Selected file: C:/Users/Asus/Documents/PhD/Projects/Insect PhD/data/Mosquito/google38.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
The uploaded image is classified as: Mosquito
Selected file: C:/Users/Asus/Documents/PhD/Projects/Insect PhD/data/Grasshopper/google5.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
The uploaded image is classified as: Mosquito
Selected file: C:/Users/Asus/Documents/PhD/Projects/Insect PhD/data/Grasshopper/google4.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
The uploaded image is classified as: Grasshopper
Selected file: C:/Users/Asus/Documents/PhD/Projects/Insect PhD/data/Dragonfly/google30.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step
The uploaded image is classified as: Mosquito
Selected file: C:/Users/Asus/Documents/PhD/Projects/Insect PhD/data/Dragonfly